In [42]:
import pickle
pkl_path = '/project/graziul/ra/ajays/whitelisted_vad_dict.pkl'
file = open(pkl_path,'rb')
vad_dict = pickle.load(file)

In [4]:
key = list(vad_dict.keys())[0]

In [44]:
import torchaudio
from torchaudio import transforms
import pickle
from speechbrain.pretrained import VAD
import os
import numpy as np

precision_list = []
recall_list = []
f1_list = []

for key in vad_dict.keys():
    waveform, sample_rate = torchaudio.load(key, normalize=True)
    transform = transforms.Resample(sample_rate, 16000)
    waveform = transform(waveform)
    title = os.path.basename(key)[:-4] 
    path = '/project/graziul/ra/anishk/VAD/Data/' + title + '.wav'
    print(path)
    torchaudio.save(path, waveform, 16000, format='wav', encoding="PCM_S", bits_per_sample=16)
    VAD = VAD.from_hparams(source="speechbrain/vad-crdnn-libriparty", savedir="pretrained_models/vad-crdnn-libriparty")
    boundaries = np.array(VAD.get_speech_segments(path))

    predictions = np.zeros(vad_dict[key]['nonsilent_slices'][-1][1])
    for n in range(boundaries.shape[0]):
        start = int(boundaries[n][0]*1000)
        end = int(boundaries[n][1]*1000)
        predictions[start:end] = 1  
    #print(predictions)

    truth = np.zeros(vad_dict[key]['nonsilent_slices'][-1][1])
    for n in range(len(vad_dict[key]['nonsilent_slices'])):
        start = vad_dict[key]['nonsilent_slices'][n][0]
        end = vad_dict[key]['nonsilent_slices'][n][1]
        truth[start:end] = 1                  
    #print(truth)

    changes = np.where(np.diff(np.sign(predictions)))[0]
    print(changes)
    
    for i in range(len(changes)-1):
        if changes[i] + 60 > changes[i+1]:
            print(changes[i],changes[i+1])

    # precision = metrics.precision_score(truth, predictions)
    # precision_list.append(precision)
    # recall = metrics.recall_score(truth, predictions)
    # recall_list.append(recall)
    # f1 = metrics.f1_score(truth, predictions)
    # f1_list.append(f1)


/project/graziul/ra/anishk/VAD/Data/201808100021-669974-27730.wav
[   6979    9388    9999   15728   24679   27868   29999   35789   43158
   44658   47859   55798   58228   60378  293308  295379  383659  390538
  391969  393868  556928  558918  676249  678949  686478  689569  691218
  702629  759999  769888  802228  805228  889999  890348  892978  895189
  899999  901578  987328  989528  991879  993158  994578 1002118 1214618
 1216048 1648769 1651858 1654738 1661668 1771688 1782608 1792338 1795798
 1798688 1803178 1804188 1809988]
/project/graziul/ra/anishk/VAD/Data/201808100051-862340-27730.wav
[   1718    5178    7999   12129   13548   19948   42828   44488   45058
   45618   59999   66138   68798   75219  264148  266518  268668  270969
  272078  274838  390008  392478  394608  396969  475659  476878  477418
  482499  491068  491808  574288  579988  654468  656828  657288  658408
  659999  669988  682518  684588  687798  693238  698098  699658  709699
  716408  719288  722689  72556

KeyboardInterrupt: 